# Data Collection

지금까지는 데이터를 쓸 때 `local` 에 있는 파일만 사용했다.  
간편하게 사용할 수 있지만 모든 데이터를 `local` 컴퓨터에 들고 올 수는 없다.  
`DB` 에서 조회하거나 `web` 에서 데이터를 수집할 때 사용할 수 있는 방법에 대해 알아보자.  

그리고 그 데이터를 정제하여 분석하기 용이한 형태로 변경해 보자!

# HTTP ?

HyperText Transfer Protocol

원격에 있는 서버와 통신하기 위한 프로토콜  
`client` 가 `요청(request)`하면  
`server` 가 `응답(response)`하는 구조를 가진다.  
`web` 에서 자료를 가져올 때 `HTTP` 를 사용하여 `client-server`간 통신을 한다.  
자료의 주소는 `url` 로 표시되며 `request parameter` 에 요청하고자 하는 정보의 조건을 명시할 수 있다.   

- https://ko.wikipedia.org/wiki/HTTP

# REST?

Representational state transfer

`web(http)` 에서 자료주소를 지정하기 위한 소프트웨어 아키텍쳐의 사실상(de facto) 표준이다.  

`HTTP` 에서는 자료의 주소를 지정하기 위해 `url` 을 사용하는데  
이 자료의 주소를 명확하고 효과적으로 나타내기 위한 방법론이다.  

이러한 주소체계를 따르는 application 을 `RESTful` 하다고 말하기도 한다.

- https://ko.wikipedia.org/wiki/REST

# JSON ?

JavaScript Object Notation

`HTTP` 를 사용하여 `web` 에서 자료를 주고 받을 때 그 자료를 표현하는 방법 중 하나이다.  
`key-value` 로 이루어져 있으며 `python` 의 `dict`와 매우 유사하다.  
`python` 에서는 `json` library 를 활용하여 손쉽게 이용가능하다.  

- https://ko.wikipedia.org/wiki/JSON

`dict`를 이용해서 `json` 문자열을 만드는 예시이다.

In [1]:
import json

d = {
    "name": "park",
    "age": 25
}

json.dumps(d)

'{"name": "park", "age": 25}'

# How to use OpenAPI ?

- 원하는 데이터를 검색
- 검색결과 확인
- 데이터 다운로드 또는 활용 신청
- https://data.go.kr/ugs/selectPublicDataUseGuideView.do#publicData_search_02

# Data we will use

일별 기온 데이터를 사용하기 위해서 `지상(종관, ASOS) 일자료 조회서비스` 를 활용한다.

- 종관기상관측 장비로 관측한 일 기상자료를 조회하는 서비스
- https://data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15059093

서버와 HTTP 통신하기 위해 `requests` 모듈을 사용한다.  

In [2]:
import requests

공공데이터포털에서 제공하는 요청주소를 url 로 설정한다.

In [3]:
url = "http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList"

공공데이터포털에서 발급해준 키를 아래에 복사 붙여넣기 해준다.  
※ 마이페이지 - 지상(종관, ASOS) 일자료 조회서비스 안에 들어가면 `service key` 를 확인할 수 있다.

In [4]:
KEY = "6qADBaxg%2BIH[~THIS IS SECRET~]"

API Key(`ServiceKey`)가 제공될때는 `URL Encode` 되어 있는데  
`urllib.parse` 모듈의 `unquote` 함수를 사용해서 `URL Decode` 시켜준다.  
이 과정을 하지 않으면 정상적인 Key 를 보낼 수 없어서 요청이 제대로 되지 않는다.  

In [5]:
from urllib.parse import unquote

decoded_key = unquote(KEY)

기타 요청에 필요한 정보를 담을 `request parameter` 를 python `dict` 형식으로 작성한다.

In [6]:
params = {
    "ServiceKey": decoded_key,
    "dataType": "JSON",
    "dataCd": "ASOS",
    "dateCd": "DAY",
    "startDt": 20210101,
    "endDt": 20210131,
    "stnIds": 159,
}

`requests` 모듈로 request parameter(`params`) 를 담아 `http get` 요청을 한 후 결과값을 response 변수에 할당한다.

In [7]:
response = requests.get(url, params=params)

response 객체의 `text` 속성을 이용해서 `http response` 텍스트를 확인해본다.  
요청할 때 `dataType` 을 `JSON` 으로 설정해서 결과값이 `json` 형식인 것으로 확인된다.

In [8]:
print(
    response.text[:100],
    "...",
    response.text[-100:]
)

{"response":{"header":{"resultCode":"00","resultMsg":"NORMAL_SERVICE"},"body":{"dataType":"JSON","it ... "sumSmlEv":"2.6","n99Rn":"","iscs":"","sumFogDur":""}]},"pageNo":1,"numOfRows":10,"totalCount":31}}}


`text` 데이터를 `json` 형식으로 `parsing` 하기 위해 `json()` 메서드를 사용한다.

In [9]:
type(response.json())

dict

In [10]:
response.json()["response"]["header"]

{'resultCode': '00', 'resultMsg': 'NORMAL_SERVICE'}

전체 데이터가 다 온건가..?

In [11]:
response_json = response.json()
body = response_json["response"]["body"]

page_no = body["pageNo"]
num_of_rows = body["numOfRows"]
total_count = body["totalCount"]

print("pageNo:", page_no)
print("numOfRows:", num_of_rows)
print("totalCount:", total_count)

pageNo: 1
numOfRows: 10
totalCount: 31


총 데이터의 개수(`total_count`)가 `31개` 인데  
한 페이지에 `10개` 씩(`num_of_rows`) 표시되고  
현재 페이지가(`page_no`)가 `1 페이지` 이면  
총 `4 페이지`(`math.ceil(total_count/num_of_rows)`)가 있음을 알 수 있다.

In [12]:
import math

math.ceil(total_count/num_of_rows)

4

그럼 `math.ceil(total_count/num_of_rows)`번을 반복해서 모두 들고 오도록 수정해보자!  
요청할 때 request parameter 로 `pageNo` 를 증가시켜줘서 모두 들고 올 수 있다.

In [13]:
for page in range(1, math.ceil(total_count/num_of_rows)+1):
    print(page)

1
2
3
4


그럼 기온데이터를 들고오는 스크립트를 함수로 만들어서 편하기 활용할 수 있게 프로그래밍 하자.  

구현방법은  
우선 요청을 하고 현재 페이지가 마지막 페이지가 아니면 다시 요청을 하고...  
이것을 반복하는 형태로 만들면 해당 기간의 모든 데이터를 들고 올 수 있을 것이다!

### ...

막상 생각을 코드로 옮기려니 막막하다!  

그래서!  
제가 구현해놨습니다!  

자세한 구현 내용이 궁금하신 분은 `api.py` 파일을 참고하세요~

In [1]:
def download_api_file_from_github():
    import requests
    content = requests.get("https://raw.githubusercontent.com/pparkddo/nano-degree/main/api.py").content
    with open("api.py", "wb") as file:
        file.write(content)

In [2]:
from os.path import isfile

if not isfile("api.py"):
    download_api_file_from_github()

from api import get_temperature_data

미리 정의된 `get_temperature_data` 메서드를 사용하여 데이터를 들고 와보자!

In [15]:
data = get_temperature_data(20201026, 20210215, 159, KEY)

정상적으로 들고온 것을 확인할 수 있다.

In [16]:
for key, value in list(data[0].items())[:15]:
    print(key, ":", value)
print("...")

stnId : 159
stnNm : 부산
tm : 2020-10-26
avgTa : 15.9
minTa : 11.7
minTaHrmt : 648
maxTa : 21.5
maxTaHrmt : 1151
mi10MaxRnHrmt : 
hr1MaxRn : 
hr1MaxRnHrmt : 
sumRn : 
maxInsWs : 10.3
maxInsWsWd : 320
maxInsWsHrmt : 137
...


# Data Preprocessing

이제 `data` 를 `pandas` 에서 다뤄보자

In [17]:
import pandas as pd

위에서 들고온 `data` 를 `DataFrame` 으로 변환시킨다.

In [18]:
df = pd.DataFrame(data)

In [19]:
df.head()

,stnId,stnNm,tm,avgTa,minTa,minTaHrmt,maxTa,maxTaHrmt,mi10MaxRnHrmt,hr1MaxRn,...,avgM05Te,avgM10Te,avgM15Te,avgM30Te,avgM50Te,sumLrgEv,sumSmlEv,n99Rn,iscs,sumFogDur
0,159,부산,2020-10-26,15.9,11.7,648,21.5,1151,,,...,18.3,21.4,22.1,20.9,20.5,3.2,4.6,,,
1,159,부산,2020-10-27,16.7,12.2,636,22.9,1306,,,...,18.3,21.2,22.0,20.8,20.5,3.4,4.9,0.0,,
2,159,부산,2020-10-28,17.3,15.4,719,22.9,1425,,0.0,...,18.4,21.0,21.9,20.8,20.5,3.0,4.3,,{비}0720-0820.,
3,159,부산,2020-10-29,15.4,10.6,656,21.9,1233,,,...,18.2,20.8,21.7,20.8,20.5,2.8,4.1,,,
4,159,부산,2020-10-30,15.0,11.4,604,21.0,1319,,,...,18.1,20.7,21.6,20.8,20.5,3.4,4.9,,,


`DataFrame` 의 `shape` `(행, 열)` 을 확인한다.

In [20]:
df.shape

(113, 60)

어떤 컬럼들이 있는지 확인해보자

In [21]:
df.columns

Index(['stnId', 'stnNm', 'tm', 'avgTa', 'minTa', 'minTaHrmt', 'maxTa',
       'maxTaHrmt', 'mi10MaxRnHrmt', 'hr1MaxRn', 'hr1MaxRnHrmt', 'sumRn',
       'maxInsWs', 'maxInsWsWd', 'maxInsWsHrmt', 'maxWs', 'maxWsWd',
       'maxWsHrmt', 'avgWs', 'hr24SumRws', 'maxWd', 'avgTd', 'minRhm',
       'minRhmHrmt', 'avgRhm', 'avgPv', 'avgPa', 'maxPs', 'maxPsHrmt', 'minPs',
       'minPsHrmt', 'avgPs', 'ssDur', 'sumSsHr', 'hr1MaxIcsrHrmt',
       'hr1MaxIcsr', 'sumGsr', 'ddMefs', 'ddMefsHrmt', 'ddMes', 'ddMesHrmt',
       'sumDpthFhsc', 'avgTca', 'avgLmac', 'avgTs', 'minTg', 'avgCm5Te',
       'avgCm10Te', 'avgCm20Te', 'avgCm30Te', 'avgM05Te', 'avgM10Te',
       'avgM15Te', 'avgM30Te', 'avgM50Te', 'sumLrgEv', 'sumSmlEv', 'n99Rn',
       'iscs', 'sumFogDur'],
      dtype='object')

필요한 컬럼만 선택하고 나머지 컬럼은 삭제해보자  
이번 실습에서는 아래에 나열된 컬럼만 사용한다.  

 - `일시(tm)`
 - `평균기온(avgTa)`
 - `최저기온(minTa)`
 - `최저기온시각(minTaHrmt)`
 - `최고기온(maxTa)`
 - `최대기온시각(maxTaHrmt)`
 - `일강수량(sumRn)`

In [22]:
columns = ["tm", "avgTa", "minTa", "minTaHrmt", "maxTa", "maxTaHrmt", "sumRn"]

df = df[columns]

In [23]:
df.head()

,tm,avgTa,minTa,minTaHrmt,maxTa,maxTaHrmt,sumRn
0,2020-10-26,15.9,11.7,648,21.5,1151,
1,2020-10-27,16.7,12.2,636,22.9,1306,
2,2020-10-28,17.3,15.4,719,22.9,1425,0.0
3,2020-10-29,15.4,10.6,656,21.9,1233,
4,2020-10-30,15.0,11.4,604,21.0,1319,


`rename` 메서드를 사용해서 컬럼명을 더 알아보기쉽게 바꿔보자.

In [24]:
column_names = {
    "tm": "day",
    "avgTa": "average_temp",
    "minTa": "min_temp",
    "minTaHrmt": "min_temp_timestamp",
    "maxTa": "max_temp",
    "maxTaHrmt": "max_temp_timestamp",
    "sumRn": "rainfall",
}

df = df.rename(columns=column_names)

In [25]:
df.head(3)

,day,average_temp,min_temp,min_temp_timestamp,max_temp,max_temp_timestamp,rainfall
0,2020-10-26,15.9,11.7,648,21.5,1151,
1,2020-10-27,16.7,12.2,636,22.9,1306,
2,2020-10-28,17.3,15.4,719,22.9,1425,0.0


`info()` 메서드로 확인한 현재 `df` 의 컬럼 상태는 아래와 같다.  
  
- 객체의 `type` 이 `DataFrame` 이다.
- `Index` 는 `RangeIndex` 를 사용하며, 113개, 0~112의 값을 가진다.
- 총 7 개의 `columns` 가 있다.
- `day`, `average_temp`, `...` 등의 컬럼이 있으며 모두 113개 `non-null` 컬럼이다.
- `dtype` 은 모두 `object` 이다.
- 현재 사용하고 있는 memory 는 6.3+ KB 이다.

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113 entries, 0 to 112
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   day                 113 non-null    object
 1   average_temp        113 non-null    object
 2   min_temp            113 non-null    object
 3   min_temp_timestamp  113 non-null    object
 4   max_temp            113 non-null    object
 5   max_temp_timestamp  113 non-null    object
 6   rainfall            113 non-null    object
dtypes: object(7)
memory usage: 6.3+ KB


`empty string("")` 의 갯수를 확인해보자

In [27]:
(df == "").sum()

day                    0
average_temp           0
min_temp               0
min_temp_timestamp     0
max_temp               0
max_temp_timestamp     0
rainfall              95
dtype: int64

 `empty string` 은 `pandas` 에서 다루기 어려우므로 `NaN` 형태로 변환한다.  

1. `NaN` 관련 메서드는 많이 제공 되지만 (`isna` 등) `empty string` 은 직접 찾아서 처리해줘야 하기 때문이다.  
2. `empty string` 을 포함한 `rainfall` 컬럼은 `강수량` 을 가지는 열이다.  
해당 열 `dtype` 으로는 `float`(실수) 타입이 올바를 것인데  
`float` 자료형에서 `null` 값을 나타내는 가장 좋은 표현방법은 `NaN` 이다.
3. `empty string` 을 포함한 열은 `dtype` 을 `float` 로 변경할 수 없다.


In [28]:
import numpy as np

df["rainfall"] = df["rainfall"].replace("", np.nan)

`rainfall` 컬럼의 `""` 값이 `np.NaN` 값으로 정상적으로 변경된 것을 알 수 있다.

In [29]:
df.head()

,day,average_temp,min_temp,min_temp_timestamp,max_temp,max_temp_timestamp,rainfall
0,2020-10-26,15.9,11.7,648,21.5,1151,NaN
1,2020-10-27,16.7,12.2,636,22.9,1306,NaN
2,2020-10-28,17.3,15.4,719,22.9,1425,0.0
3,2020-10-29,15.4,10.6,656,21.9,1233,NaN
4,2020-10-30,15.0,11.4,604,21.0,1319,NaN


In [30]:
df.isna().sum()

day                    0
average_temp           0
min_temp               0
min_temp_timestamp     0
max_temp               0
max_temp_timestamp     0
rainfall              95
dtype: int64

`.astype()` 메서드를 사용해서 각 컬럼에 맞는 `dtype` 으로 변경해준다.

 - `day` : `Timestamp`
 - `average_temp` : `float`
 - `min_temp` : `float`
 - `min_temp_timestamp` : `Timestamp`
 - `max_temp` : `float`
 - `max_temp_timestamp` : `Timestamp`
 - `rainfall` : `float`

우선 `float` 타입을 가질 컬럼들의 `dtype` 을 변경해준다.

In [31]:
float_type_columns = [
    "average_temp",
    "min_temp",
    "max_temp",
    "rainfall",
]

df[float_type_columns] = df[float_type_columns].astype(float)

`Timestamp` 타입을 가질 컬럼들의 `dtype` 을 변경해준다.

같은 `Timestamp` 컬럼이라도 기존 양식(`format`)에 따라 따로 적용해줘야한다.  

- `day` : `%Y-%m-%d` (ex. 20190101)
- `min_temp_timestamp` : `%H%M` (ex. 0648)
- `max_temp_timestamp` : `%H%M` (ex. 0648)  

date time format: https://docs.python.org/ko/3/library/datetime.html#strftime-and-strptime-format-codes

`day` 열을 `%Y-%m-%d` 형식에 맞게 `Timestamp` `dtype` 으로 변경한다.

In [32]:
df["day"] = pd.to_datetime(df["day"], format="%Y-%m-%d")

print(df["day"])

0     2020-10-26
1     2020-10-27
2     2020-10-28
3     2020-10-29
4     2020-10-30
         ...    
108   2021-02-11
109   2021-02-12
110   2021-02-13
111   2021-02-14
112   2021-02-15
Name: day, Length: 113, dtype: datetime64[ns]


`*_timestamp` 열의 `dtype` 을 `Timestamp` 형식으로 변경시켜보자

In [33]:
df["min_temp_timestamp"] = df["min_temp_timestamp"].str.pad(4, side="left", fillchar="0")
df["max_temp_timestamp"] = df["max_temp_timestamp"].str.pad(4, side="left", fillchar="0")

In [34]:
df.head()

,day,average_temp,min_temp,min_temp_timestamp,max_temp,max_temp_timestamp,rainfall
0,2020-10-26,15.9,11.7,0648,21.5,1151,NaN
1,2020-10-27,16.7,12.2,0636,22.9,1306,NaN
2,2020-10-28,17.3,15.4,0719,22.9,1425,0.0
3,2020-10-29,15.4,10.6,0656,21.9,1233,NaN
4,2020-10-30,15.0,11.4,0604,21.0,1319,NaN


`timestamp` 의 앞의 두글자는 `hour` 를 나타낸다.

In [35]:
df["min_temp_timestamp"].str[:2]

0      06
1      06
2      07
3      06
4      06
       ..
108    06
109    07
110    07
111    05
112    23
Name: min_temp_timestamp, Length: 113, dtype: object

`timestamp` 의 뒤의 두글자는 `minute` 을 나타낸다.

In [36]:
df["min_temp_timestamp"].str[2:]

0      48
1      36
2      19
3      56
4      04
       ..
108    47
109    02
110    27
111    35
112    55
Name: min_temp_timestamp, Length: 113, dtype: object

`timestamp` 에서 각각 시간정보(`hour`, `minute`)를 뽑아내서 `int` 형으로 변환한 다음  
시간을 표현하기위한 자료형인 `pd.Timedelta` 형식으로 변경해준다.

In [37]:
min_hour = pd.to_timedelta(df["min_temp_timestamp"].str[:2].astype(int), unit="h")

print(min_hour)

0     06:00:00
1     06:00:00
2     07:00:00
3     06:00:00
4     06:00:00
        ...   
108   06:00:00
109   07:00:00
110   07:00:00
111   05:00:00
112   23:00:00
Name: min_temp_timestamp, Length: 113, dtype: timedelta64[ns]


In [38]:
min_minute = pd.to_timedelta(df["min_temp_timestamp"].str[2:].astype(int), unit="m")

print(min_minute)

0     00:48:00
1     00:36:00
2     00:19:00
3     00:56:00
4     00:04:00
        ...   
108   00:47:00
109   00:02:00
110   00:27:00
111   00:35:00
112   00:55:00
Name: min_temp_timestamp, Length: 113, dtype: timedelta64[ns]


`day`, `hour` 과 `minute` 정보를 모두 합쳐서 온전한 `Timestamp` 열로 변환한다.

In [39]:
df["min_temp_timestamp"] = df["day"] + min_hour + min_minute

print(df["min_temp_timestamp"])

0     2020-10-26 06:48:00
1     2020-10-27 06:36:00
2     2020-10-28 07:19:00
3     2020-10-29 06:56:00
4     2020-10-30 06:04:00
              ...        
108   2021-02-11 06:47:00
109   2021-02-12 07:02:00
110   2021-02-13 07:27:00
111   2021-02-14 05:35:00
112   2021-02-15 23:55:00
Name: min_temp_timestamp, Length: 113, dtype: datetime64[ns]


마찬가지로 `max_temp_timestamp` 도 `Timestamp` `dtype` 으로 변경해준다.


In [40]:
df["max_temp_timestamp"] = (
    df["day"]
    + pd.to_timedelta(df["max_temp_timestamp"].str[:2].astype(int), unit="h")
    + pd.to_timedelta(df["max_temp_timestamp"].str[2:].astype(int), unit="m")
)

In [41]:
df.head()

,day,average_temp,min_temp,min_temp_timestamp,max_temp,max_temp_timestamp,rainfall
0,2020-10-26,15.9,11.7,2020-10-26 06:48:00,21.5,2020-10-26 11:51:00,NaN
1,2020-10-27,16.7,12.2,2020-10-27 06:36:00,22.9,2020-10-27 13:06:00,NaN
2,2020-10-28,17.3,15.4,2020-10-28 07:19:00,22.9,2020-10-28 14:25:00,0.0
3,2020-10-29,15.4,10.6,2020-10-29 06:56:00,21.9,2020-10-29 12:33:00,NaN
4,2020-10-30,15.0,11.4,2020-10-30 06:04:00,21.0,2020-10-30 13:19:00,NaN


이렇게 모든 열의 `dtype` 이 올바르게 지정되었다.  
올바른 `dtype` 을 가졌기 때문에 이후에 분석에 있어서 매우 용이할 것 이다.  

예를 들면 `min_temp_timestamp` 에 하루씩 빼주는 연산을 하려면  
`str` 타입일때는 매우 힘들지만 `Timestamp` 타입일 때는 아래와 같이 매우 간편하게 할 수 있다.  
```python
df["min_temp_timestamp"] - pd.Timedelta(1, unit="d")
```  
  
컬럼별로 올바른 데이터타입(`dtype`) 을 지정해주는 것은 기본이다!

마지막으로 `rainfall` 컬럼에 `NaN` 데이터를 `0` 으로 치환해보자

In [42]:
df["rainfall"] = df["rainfall"].fillna(0)

In [43]:
df.head()

,day,average_temp,min_temp,min_temp_timestamp,max_temp,max_temp_timestamp,rainfall
0,2020-10-26,15.9,11.7,2020-10-26 06:48:00,21.5,2020-10-26 11:51:00,0.0
1,2020-10-27,16.7,12.2,2020-10-27 06:36:00,22.9,2020-10-27 13:06:00,0.0
2,2020-10-28,17.3,15.4,2020-10-28 07:19:00,22.9,2020-10-28 14:25:00,0.0
3,2020-10-29,15.4,10.6,2020-10-29 06:56:00,21.9,2020-10-29 12:33:00,0.0
4,2020-10-30,15.0,11.4,2020-10-30 06:04:00,21.0,2020-10-30 13:19:00,0.0


전처리가 끝난 데이터를 `csv` 파일 형식으로 저장하자!

In [44]:
df.to_csv("preprocessing.csv")